In [1]:
import os
import json
import logging
import numpy as np
from tqdm import tqdm
from utils.load_dataset import load_dataset

with open("./config.json") as f:
    config = json.load(f)

logger = logging.getLogger("")
dataset_name = 'cifar10'
arch = 'resnet18'
num_augs = 2

dataset = load_dataset(
    dataset=dataset_name,
    train_batch_size=128,
    test_batch_size=128,
    val_split=0.0,
    augment=False,
    shuffle=False,
    root_path=config['data_dir'],
    random_seed=0,
    mean=[0, 0, 0],
    std=[1, 1, 1],
    logger=logger,
)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [2]:
def get_scores_and_masks(seeds, dataset, method, num_augs):
    h = 0.001
    num_seeds = len(seeds)
    aug_scores_file_name = {
        'curv_zo' : "curvature_scores_zo_{}_{}_{}_{}_tid{}.pt",
        'curv' : "curvature_scores_{}_{}_{}_{}_tid{}.pt",
        'prob' : "prob_{}_{}_{}_tid{}.pt",
        'loss' : "losses_{}_{}_{}_tid{}.pt",
        'mentr' : "m_entropy_{}_{}_{}_tid{}.pt",
        'loss_g' : "loss_g_{}_{}_{}_tid{}.pt",
    }

    score_seeds_v_seed = np.zeros((num_augs, dataset.train_length, num_seeds))
    mask_seeds_v_seed = np.zeros((dataset.train_length, num_seeds))
    start_seed = seeds[0]
    for exp_idx in seeds:
        array_idx = exp_idx - start_seed
        augs = 1 if method == 'mentr' else num_augs
        for aug_idx in range(augs):
            if 'curv' in aug_scores_file_name[method]:
                file_name = aug_scores_file_name[method].format(dataset.name, arch, exp_idx, h, aug_idx)
            else:
                file_name = aug_scores_file_name[method].format(dataset.name, arch, exp_idx, aug_idx)

            absolute_file_name = os.path.join(config['precomputed_scores_dir'], dataset.name, file_name)
            scores = np.load(absolute_file_name)['data']
            score_seeds_v_seed[aug_idx, :, array_idx] = scores

        if dataset == 'imagenet':
            array = np.load(f'/path/imagenet-resnet50/{0.7}/{exp_idx}/aux_arrays.npz')
            mask_idxs = array['subsample_idx']
            mask_seeds_v_seed[mask_idxs, array_idx] = 1
        else:
            mask_idxs = np.load(os.path.join(config['subset_idxs_dir'], f"{exp_idx}.npy"))
            mask_seeds_v_seed[mask_idxs, array_idx] = 1

    return score_seeds_v_seed, mask_seeds_v_seed

def get_score_dist_params(scores, masks, num_augs):
    means_in = []
    means_out = []
    var_in = []
    var_out = []
    for aug in range(num_augs):
        in_scores = np.where(masks == 1, scores[aug], np.NaN)
        out_scores = np.where(masks == 0, scores[aug], np.NaN)
        means_in.append(np.nanmean(in_scores, axis=1))
        means_out.append(np.nanmean(out_scores, axis=1))
        var_in.append(np.nanvar(in_scores, axis=1))
        var_out.append(np.nanvar(out_scores, axis=1))

    return {
        'means_in': np.row_stack(means_in),
        'means_out': np.row_stack(means_out),
        'var_in': np.row_stack(var_in),
        'var_out': np.row_stack(var_out)
    }


In [19]:
scores = {}
masks = {}
dist_params = {}
train_end_seed = 64
train_seeds = list(range(train_end_seed))
test_seeds = list(range(train_end_seed, train_end_seed + 3))
eps = 1e-22

In [20]:
def log_err(x):
    return np.log(x + eps)

def logit_scale(x):
    x = x / (1 - x + eps)
    return log_err(x)

def nll(curv_score, mean, var):
    ll = - ( ( (curv_score - mean)**2) / (2 * (var ** 2) ) ) -0.5 * np.log(var ** 2) - 0.5 * np.log(2 * np.pi)
    return -ll

def likelihood(curv_score, mean, var):
    nll = - ( ( (curv_score - mean)**2) / (2 * (var ** 2) ) ) - 0.5 * np.log(var ** 2) - 0.5 * np.log(2 * np.pi)
    likelihood_val = np.exp(nll)
    return likelihood_val

def get_likelihood_ratio(test_scores, dist_params, num_augs, train_scores, train_masks):
    in_likelihood = np.zeros((test_scores.shape[1]))
    out_likelihood = np.zeros_like(in_likelihood)
    for aug in range(num_augs):
        test_score_aug = test_scores[aug, :, 0]
        in_likelihood += likelihood(
            test_score_aug, 
            dist_params['means_in'][aug], 
            dist_params['var_in'][aug] + eps)

        out_likelihood += likelihood(
            test_score_aug, 
            dist_params['means_out'][aug], 
            dist_params['var_out'][aug] + eps)

    likelihood_ratio = in_likelihood / (out_likelihood + eps)
    return likelihood_ratio

def get_likelihood_ratio_c(test_scores, dist_params, num_augs, train_scores, train_masks):
    in_likelihood = np.zeros((test_scores.shape[1]))
    out_likelihood = np.zeros_like(in_likelihood)
    for aug in range(num_augs):
        test_score_aug = test_scores[aug, :, 0]
        in_likelihood += likelihood(
            test_score_aug, 
            dist_params['means_in'][aug], 
            1)

        out_likelihood += likelihood(
            test_score_aug, 
            dist_params['means_out'][aug], 
            1)

    likelihood_ratio = in_likelihood / (out_likelihood + eps)
    return likelihood_ratio

def get_nll_ratio_c(test_scores, dist_params, num_augs, train_scores, train_masks):
    in_likelihood = np.zeros((test_scores.shape[1]))
    out_likelihood = np.zeros_like(in_likelihood)
    for aug in range(num_augs):
        test_score_aug = test_scores[aug, :, 0]
        in_likelihood += nll(
            test_score_aug, 
            dist_params['means_in'][aug], 
            1)

        out_likelihood += nll(
            test_score_aug, 
            dist_params['means_out'][aug], 
            1)

    likelihood_ratio = in_likelihood - out_likelihood
    return -likelihood_ratio

def get_nll_ratio(test_scores, dist_params, num_augs, train_scores, train_masks):
    in_likelihood = np.zeros((test_scores.shape[1]))
    out_likelihood = np.zeros_like(in_likelihood)
    for aug in range(num_augs):
        test_score_aug = test_scores[aug, :, 0]
        in_likelihood += nll(
            test_score_aug, 
            dist_params['means_in'][aug], 
            dist_params['var_in'][aug] + 1e-32)

        out_likelihood += nll(
            test_score_aug, 
            dist_params['means_out'][aug], 
            dist_params['var_in'][aug] + 1e-32)

    likelihood_ratio = in_likelihood - out_likelihood
    return -likelihood_ratio

def get_identity(test_scores, dist_params, num_augs, train_scores, train_masks):
    return -test_scores[0]

def get_mast(test_scores, dist_params, num_augs, train_scores, train_masks):
    aug = 0
    threshold = (dist_params['means_in'][aug] +  dist_params['means_out'][aug]) / 2 
    return -(test_scores[aug, :, 0] - threshold)

def get_mast_offline(test_scores, dist_params, num_augs, train_scores, train_masks):
    aug = 0
    threshold = dist_params['means_out'][aug]
    return -(test_scores[aug, :, 0] - threshold)

def get_ye_et_el_attack_r(test_scores, dist_params, num_augs, train_scores, train_masks):
    aug = 0
    alpha = 1e-4
    out_scores = np.where(train_masks == 0, train_scores[aug], np.NaN)
    threshold = np.nanpercentile(out_scores, alpha * 100, axis=1)
    return -(test_scores[aug, :, 0] - threshold)

def get_class_based(test_scores, dist_params, num_augs, train_scores, train_masks):
    # This is handled during roc calculations
    return -test_scores[0, :, 0]

def get_loss_count(test_scores, dist_params, num_augs, train_scores, train_masks):
    in_scores = np.where(train_masks == 1, train_scores[0], np.NaN)
    threshold = np.nanmean(in_scores)
    pred = (test_scores[:, :, 0] < threshold).sum(0) / num_augs
    return pred

methods = [
    ('curv_zo', log_err, get_nll_ratio, 'Curv ZO NLL', num_augs, train_seeds), 
    ('curv_zo', log_err, get_likelihood_ratio, 'Curv ZO LR', num_augs, train_seeds), 
    ('prob', logit_scale, get_likelihood_ratio, 'Carlini et al.', num_augs, train_seeds),
    ('loss', lambda x: x, get_identity, 'Yeom et al.', 1, [0]),
    ('loss', lambda x: x, get_mast, 'Sablayrolles et al.', 1, train_seeds),
    ('loss', lambda x: x, get_mast_offline, 'Watson et al.', 1, train_seeds),
    ('loss', lambda x: x, get_ye_et_el_attack_r, 'Ye et al.', 1, train_seeds),
    ('mentr', lambda x: x, get_class_based, 'Song et al.', 1, train_seeds),
]


In [21]:
for info_type, score_func, _, method_name, augs, seeds in tqdm(methods):
    if method_name in scores:
        continue
    
    score, mask = get_scores_and_masks(
        seeds=seeds, 
        dataset=dataset,
        method=info_type,
        num_augs=augs)

    score = score_func(score)
    params = get_score_dist_params(score, mask, augs)
    scores[method_name] = score
    masks[method_name] = mask
    dist_params[method_name] = params

 38%|███▊      | 3/8 [00:03<00:05,  1.12s/it]/tmp/ipykernel_16868/2255348333.py:47: RuntimeWarning: Mean of empty slice
  means_in.append(np.nanmean(in_scores, axis=1))
/tmp/ipykernel_16868/2255348333.py:48: RuntimeWarning: Mean of empty slice
  means_out.append(np.nanmean(out_scores, axis=1))
/tmp/ipykernel_16868/2255348333.py:49: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var_in.append(np.nanvar(in_scores, axis=1))
/tmp/ipykernel_16868/2255348333.py:50: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var_out.append(np.nanvar(out_scores, axis=1))
100%|██████████| 8/8 [00:06<00:00,  1.30it/s]


In [22]:
test_scores = {}
test_masks = {}

for info_type, score_func, _, method_name, augs, _ in tqdm(methods):
    test_score, test_mask = get_scores_and_masks(
        seeds=test_seeds, 
        dataset=dataset,
        method=info_type,
        num_augs=augs)

    test_scores[method_name] = score_func(test_score)
    test_masks[method_name] = test_mask


100%|██████████| 8/8 [00:00<00:00, 35.81it/s]


In [23]:
from sklearn.metrics import roc_auc_score, roc_curve, balanced_accuracy_score

auroc_all = {}
bal_acc_all = {}

for test_seed_idx, test_seed in enumerate(test_seeds):
    print(f"=================== Seed {test_seed} ======================")
    for idx, (info_type, _, pred_func, method_name, augs, _) in enumerate(methods):
        y_pred = pred_func(
            test_scores[method_name][..., test_seed_idx, np.newaxis],
            dist_params[method_name],
            augs,
            scores[method_name],
            masks[method_name])

        y_true = test_masks[method_name][..., test_seed_idx, np.newaxis]
        if method_name == 'Song et al.':
            labels_file_name = os.path.join(
                config['precomputed_scores_dir'], 
                dataset.name, 
                f"true_labels_cifar10_{train_end_seed}.pt")
            labels = np.load(labels_file_name)['data']
            auc = 0 
            n_classes = dataset.num_classes
            balanced_accuracy = 0
            for label in range(n_classes):
                indices = np.where(labels == label)[0]
                # Compute ROC curve for each class
                fpr, tpr, thr = roc_curve(y_true[indices], y_pred[indices])
                auc += roc_auc_score(y_true, y_pred)
                
                # Find the optimal threshold: where the sum of FPR and TPR is closest to 1
                optimal_idx = np.argmin(np.abs(fpr + tpr - 1))
                optimal_threshold = thr[optimal_idx]
                # Binarize predictions based on the optimal threshold
                y_pred_binarized = (y_pred >= optimal_threshold).astype(int)

                # Calculate Balanced Accuracy
                balanced_accuracy += balanced_accuracy_score(y_true, y_pred_binarized)
            
            auc = auc / n_classes
            balanced_accuracy = balanced_accuracy / n_classes

        else:
            fpr, tpr, thr = roc_curve(y_true, y_pred, pos_label=1)
            auc = roc_auc_score(y_true, y_pred)

            # Find the optimal threshold: where the sum of FPR and TPR is closest to 1
            # "Balanced accuracy is symmetric. That is, the metric
            # assigns equal cost to false-positives and to false-negatives."
            # - LiRA https://arxiv.org/pdf/2112.03570.pdf
            optimal_idx = np.argmin(np.abs(fpr + tpr - 1))
            optimal_threshold = thr[optimal_idx]

            # Binarize predictions based on the optimal threshold
            y_pred_binarized = (y_pred >= optimal_threshold).astype(int)

            # Calculate Balanced Accuracy
            balanced_accuracy = balanced_accuracy_score(y_true, y_pred_binarized)
        
        print(f"AUC {method_name}: {auc*100:.2f}, Acc {balanced_accuracy * 100:.2f}")
        if method_name in auroc_all:
            auroc_all[method_name] += [auc]
            bal_acc_all[method_name] += [balanced_accuracy]
        else:
            auroc_all[method_name] = [auc]
            bal_acc_all[method_name] = [balanced_accuracy]


=================== Seed 64 ======================
AUC Curv ZO NLL: 70.30, Acc 62.97
AUC Curv ZO LR: 59.44, Acc 55.24
AUC Carlini et al.: 61.99, Acc 58.49
AUC Yeom et al.: 61.12, Acc 55.97
AUC Sablayrolles et al.: 61.28, Acc 56.48
AUC Watson et al.: 58.42, Acc 54.78
AUC Ye et al.: 68.39, Acc 60.32
AUC Song et al.: 61.17, Acc 56.10
=================== Seed 65 ======================
AUC Curv ZO NLL: 67.19, Acc 60.85
AUC Curv ZO LR: 58.34, Acc 54.80
AUC Carlini et al.: 61.27, Acc 57.79
AUC Yeom et al.: 59.39, Acc 54.84
AUC Sablayrolles et al.: 62.57, Acc 57.40
AUC Watson et al.: 59.70, Acc 55.63
AUC Ye et al.: 65.55, Acc 58.43
AUC Song et al.: 59.23, Acc 54.78
=================== Seed 66 ======================
AUC Curv ZO NLL: 69.37, Acc 62.07
AUC Curv ZO LR: 59.10, Acc 55.06
AUC Carlini et al.: 62.12, Acc 58.53
AUC Yeom et al.: 60.80, Acc 55.90
AUC Sablayrolles et al.: 60.66, Acc 56.07
AUC Watson et al.: 57.62, Acc 54.19
AUC Ye et al.: 67.97, Acc 60.11
AUC Song et al.: 60.86, Acc 56.02


In [24]:
for k, v in auroc_all.items():
    auc_for_k = np.array(v)
    bal_acc_for_k = np.array(bal_acc_all[k])

    print(f"{k}, auroc {auc_for_k.mean() * 100:.2f} $\pm$ {auc_for_k.std()* 100:.2f}, bal {bal_acc_for_k.mean()* 100:.2f} $\pm$ {bal_acc_for_k.std()* 100:.2f}") 

Curv ZO NLL, auroc 68.95 $\pm$ 1.31, bal 61.96 $\pm$ 0.87
Curv ZO LR, auroc 58.96 $\pm$ 0.46, bal 55.04 $\pm$ 0.18
Carlini et al., auroc 61.79 $\pm$ 0.37, bal 58.27 $\pm$ 0.34
Yeom et al., auroc 60.44 $\pm$ 0.75, bal 55.57 $\pm$ 0.52
Sablayrolles et al., auroc 61.50 $\pm$ 0.79, bal 56.65 $\pm$ 0.56
Watson et al., auroc 58.58 $\pm$ 0.86, bal 54.86 $\pm$ 0.59
Ye et al., auroc 67.30 $\pm$ 1.25, bal 59.62 $\pm$ 0.84
Song et al., auroc 60.42 $\pm$ 0.85, bal 55.63 $\pm$ 0.61
